In [7]:
import serial

In [4]:
import ultralytics
ultralytics.checks()
import cv2
import pytesseract
import os
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator
import serial
#!git clone https://github.com/opencv/opencv

#weights_path = r"C:\Users\engmo\OneDrive\Desktop\upload\try_python\weights_model_in_YOLOv8n\weight_YOLOv8n\weights\best.pt"
#weights_path=r"C:\Users\engmo\OneDrive\Desktop\upload\try_python\weights_model_in_YOLOV8S\weights\best.pt" #one
weights_path=r"C:\Users\engmo\OneDrive\Desktop\upload\try_python\weights_model_in_YOLOv8n\weight_YOLOv8n\weights\best.pt"


# Define the path to the YOLOv8 class names file
names_path = r"C:/Users/engmo/OneDrive/Desktop/try_python/class_name.txt"

# Load the YOLOv8 model
model = YOLO(weights_path)

# Configure Tesseract OCR engine
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

cap = cv2.VideoCapture(0)
cap.set(3, 480)
cap.set(4, 640)
#capx=cv2.VideoCapture(1)
#capx=capx.set(3,480)
#capx=capx.set(4,640)

while True:
    _, frame = cap.read()#deh zi image read bas be2ra cv2.videocapture
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#bnhawek nb BGR TO RGB
    resized_image = cv2.resize(img, (640, 640))

    results = model.predict(resized_image, show=True)#RGB

    for result in results:
        
        annotator = Annotator(frame)#BGR
        
        boxes = result.boxes
        for box in boxes:
            
            cords = box.xyxy[0].tolist()# AMAKEN EL SIDES OF BOUNDING BOX(TOP,LEFT,BOTTOM,RIGHT)
            class_id = box.cls[0].item()
            #TO LIST &ITEM TO SEE RESULTS IN A FORMAT NOT LIKE THE TENSOR FLOW FORMAT
            conf = box.conf[0].item()
            area = (cords[2] - cords[0]) * (cords[3] - cords[1])
            max_area=640*480
            limit=0.8*max_area 
            if (area>limit):
             print("danger zone")
            print("Object type:", model.names[int(class_id)])
            print("Coordinates:", cords)
            print("Probability:", conf)
            print("Area:", area)
            annotator.box_label(cords, model.names[int(class_id)])
            
            # If the detected object is a speed limit sign, extract the region of interest and read the speed limit number using OCR AND ALSO CONVERT TO GREY THEN ADJUST THRESHOLD &CHOSSE PSM =9 (TEXT IN A CIRCLE CASE)
            if model.names[int(class_id)] == 'speed_limit':
                roi = img[cords[1]:cords[3], cords[0]:cords[2]]
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                #threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
                threshold = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C ,cv2.THRESH_BINARY,85,30)#85 bounding box sizw w 30 dh parameter MAKES IMAGE LIKE MAGIC COLOR OF CAM SCANNER
                text = pytesseract.image_to_string(threshold, config=f'--oem 3 --psm {9} outputbase digits').replace('\n','')# threshold new image PSM =9 (CASE OF TEXT IN A CIRCLE)
               #x=int(speed_limit_number)
                #print("Speed limit number:", speed_limit_number)
                print(f" {text}")                
                number=int (f"{text}") #CONVERT TEXT TO NUMBER TO COMPARE THE NUMBER WITH ACTUALL SPEED OF CAR FOR FUTURE WORK
                print (number)
                annotator.text((cords[0], cords[1]-20), f" {text}")
            
    frame = annotator.result()  
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.20  Python-3.10.9 torch-2.0.1+cpu CPU
Setup complete  (16 CPUs, 15.7 GB RAM, 339.4/474.7 GB disk)


Ultralytics YOLOv8.0.20  Python-3.10.9 torch-2.0.1+cpu CPU
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


KeyboardInterrupt: 